# Код для разметки угловых точек номера

In [ ]:
import cv2
import os
import glob
import tkinter as tk
from tkinter import filedialog
import shutil

class ImageMarker:
    def __init__(self):
        self.images = []
        self.current_image_index = 0
        self.points = []
        self.scale_factor = 3  # Увеличиваем изображение в 3 раза
        self.annotated_images = set()  # Set to store annotated image paths
        self.load_images()

    def load_images(self):
        root = tk.Tk()
        root.withdraw()
        folder_path = r'F:\grozny\labeling'

        if not folder_path:
            print("Папка не выбрана. Завершение программы.")
            return

        self.images = glob.glob(os.path.join(folder_path, '*.jpg')) + glob.glob(os.path.join(folder_path, '*.png'))
        self.images.sort()

        if not self.images:
            print(f"Не найдено изображений в папке: {folder_path}")
            return

        print(f"Найдено {len(self.images)} изображений в папке: {folder_path}")

        self.output_folder = r'F:\grozny\result'
        self.img_out = "img_out"
        if not self.output_folder:
            print("Папка для сохранения меток не выбрана. Завершение программы.")
            return

    def save_points(self):
        image_name = os.path.basename(self.images[self.current_image_index])
        txt_file_path = os.path.join(self.output_folder, f"{os.path.splitext(image_name)[0]}.txt")
        with open(txt_file_path, 'w') as file:
            for point in self.points:
                # Масштабируем координаты обратно перед сохранением
                original_point = (point[0] // self.scale_factor, point[1] // self.scale_factor)
                file.write(f"{original_point[0]}, {original_point[1]}\n")
        self.annotated_images.add(self.images[self.current_image_index])  # Add annotated image to set

    def click_event(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            if len(self.points) < 4:
                self.points.append((x, y))
                self.display_image()
            if len(self.points) == 4:
                self.save_points()

    def display_image(self):
        image = self.scaled_image.copy()
        for point in self.points:
            cv2.circle(image, point, 5, (0, 0, 255), -1)
            cv2.putText(image, f"{point}", point, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)
        cv2.imshow("Image", image)

    def run(self):
        if not self.images or not self.output_folder:
            return

        cv2.namedWindow("Image")
        cv2.setMouseCallback("Image", self.click_event)

        while True:
            self.original_image = cv2.imread(self.images[self.current_image_index])
            # Масштабируем изображение
            width = int(self.original_image.shape[1] * self.scale_factor)
            height = int(self.original_image.shape[0] * self.scale_factor)
            self.scaled_image = cv2.resize(self.original_image, (width, height))
            self.points = []
            self.display_image()
            key = cv2.waitKey(0)

            if key == ord('d') and self.current_image_index < len(self.images) - 1:
                self.current_image_index += 1
            elif key == ord('a') and self.current_image_index > 0:
                self.current_image_index -= 1
            elif key == ord('p'):
                break

        # After marking session ends
        cv2.destroyAllWindows()

        # Move or delete annotated images
        for img_path in self.annotated_images:
            if os.path.exists(img_path):
                shutil.move(img_path, self.img_out)  # Move to output folder
            # Optionally delete the original image after moving
            # os.remove(img_path)

if __name__ == "__main__":
    ImageMarker().run()


# Код выравнивания изображений методом перспективного преобразования

## Для выравнивания используются размеченные нами угловые точки номеров

In [ ]:
import cv2
import numpy as np
import os

def load_corners_from_file(file_path):
    with open(file_path, 'r') as file:
        corners = []
        for line in file:
            x, y = map(float, line.strip().split(','))
            corners.append((x, y))
        return np.array(corners, dtype="float32")

def align_image(image, corners):
    # Упорядочивание угловых точек в порядке: верхний левый, верхний правый, нижний правый, нижний левый
    rect = np.zeros((4, 2), dtype="float32")
    s = corners.sum(axis=1)
    rect[0] = corners[np.argmin(s)]
    rect[2] = corners[np.argmax(s)]

    diff = np.diff(corners, axis=1)
    rect[1] = corners[np.argmin(diff)]
    rect[3] = corners[np.argmax(diff)]

    # Определение ширины и высоты нового изображения
    (tl, tr, br, bl) = rect
    widthA = np.linalg.norm(br - bl)
    widthB = np.linalg.norm(tr - tl)
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.linalg.norm(tr - br)
    heightB = np.linalg.norm(tl - bl)
    maxHeight = max(int(heightA), int(heightB))

    # Новое выровненное изображение
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")

    # Преобразование перспективы
    M = cv2.getPerspectiveTransform(rect, dst)
    aligned = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    return aligned

# Пути к папкам
images_folder = r'F:\grozny\beeline\train_borders\images'
labels_folder = r'F:\grozny\beeline\train_borders\labels'
aligned_folder = r'F:\grozny\beeline\train_borders\align'
# Создание папки для выровненных изображений, если её нет
os.makedirs(aligned_folder, exist_ok=True)

# Обработка всех изображений в папке
for image_filename in os.listdir(images_folder):
    if image_filename.endswith(('.jpg', '.png', '.jpeg')):
        # Формирование путей к изображениям и текстовым файлам
        image_path = os.path.join(images_folder, image_filename)
        label_path = os.path.join(labels_folder, os.path.splitext(image_filename)[0] + '.txt')

        # Загрузка изображения и угловых точек
        image = cv2.imread(image_path)
        corners = load_corners_from_file(label_path)

        # Выравнивание изображения
        aligned_image = align_image(image, corners)

        # Сохранение выровненного изображения
        aligned_image_path = os.path.join(aligned_folder, image_filename)
        cv2.imwrite(aligned_image_path, aligned_image)

print("Все изображения обработаны и сохранены в папке 'align'.")

# Код извлечения номера региона для обучения сети

In [1]:
import os
from PIL import Image

# Папки с исходными изображениями и для сохранения
source_folder = r"F:\grozny\beeline\train_borders\align"
destination_folder = r"F:\grozny\beeline\find_region_type"

# Проверка наличия папки назначения, если её нет - создаём
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Нормализованные координаты извлекаемой области (значения от 0 до 1)
# Пример: извлечение центральной части изображения
relative_coordinates = (0.71, 0.05, 0.96, 0.7)

# Проход по всем файлам в исходной папке
for filename in os.listdir(source_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        # Путь к текущему изображению
        img_path = os.path.join(source_folder, filename)
        
        # Открываем изображение
        img = Image.open(img_path)
        width, height = img.size
        
        # Рассчитываем абсолютные координаты на основе нормализованных
        left = int(relative_coordinates[0] * width)
        top = int(relative_coordinates[1] * height)
        right = int(relative_coordinates[2] * width)
        bottom = int(relative_coordinates[3] * height)
        
        # Обрезаем изображение
        cropped_img = img.crop((left, top, right, bottom))
        
        # Сохраняем обрезанное изображение в папку назначения
        cropped_img.save(os.path.join(destination_folder, filename))

print("Готово!")


Готово!


# Код разметки извлеченных фрагментов номера региона

In [8]:
import os
import cv2

# Папка с изображениями для классификации
source_folder = r"F:\grozny\beeline\find_region_type"

# Папка для сохранения классифицированных изображений
destination_folder = r"F:\grozny\beeline\classification_region_type"
class_2_folder = os.path.join(destination_folder, '2')
class_3_folder = os.path.join(destination_folder, '3')

# Создание папок для каждого класса, если они не существуют
os.makedirs(class_2_folder, exist_ok=True)
os.makedirs(class_3_folder, exist_ok=True)

# Размер для масштабирования изображений
resize_dim = (120, 60)

# Получение списка всех файлов в исходной папке
files = [f for f in os.listdir(source_folder) if f.endswith(('.jpg', '.jpeg', '.png'))][330:455]

for filename in files:
    img_path = os.path.join(source_folder, filename)
    img = cv2.imread(img_path)
    
    if img is None:
        print(f"Не удалось открыть изображение {img_path}")
        continue

    # Масштабирование изображения
    resized_img = cv2.resize(img, resize_dim)

    # Показ изображения
    cv2.imshow('Image', resized_img)

    key = cv2.waitKey(0)  # Ожидание нажатия клавиши

    # Копирование изображения в соответствующую папку в зависимости от нажатой клавиши
    if key == ord('f'):
        destination_path = os.path.join(class_2_folder, filename)
        cv2.imwrite(destination_path, resized_img)
        print(f"{filename} сохранено в класс 2")
    elif key == ord('j'):
        destination_path = os.path.join(class_3_folder, filename)
        cv2.imwrite(destination_path, resized_img)
        print(f"{filename} сохранено в класс 3")
    else:
        print(f"{filename} не классифицировано, пропускается")

    cv2.destroyAllWindows()  # Закрыть окно изображения

print("Классификация завершена!")


546fe58c4726bf96.jpg сохранено в класс 3
54b37d1dcaa9904d.jpg сохранено в класс 2
54e17facb117450a.jpg сохранено в класс 3
54fbca9b449b6a98.jpg сохранено в класс 3
5501279f386334ea.jpg сохранено в класс 3
551ca2259c57a8d4.jpg сохранено в класс 3
5568a6b77639cfea.jpg сохранено в класс 3
55a834e4114cda85.jpg сохранено в класс 3
55c567b373af4755.jpg сохранено в класс 2
55d491d432298b2f.jpg сохранено в класс 2
563ec928a2c398a1.jpg сохранено в класс 3
56fb003ffded4de3.jpg сохранено в класс 2
570d65df46a8af32.jpg сохранено в класс 2
5742e66a9afb6b05.jpg сохранено в класс 3
57854495d0d25582.jpg сохранено в класс 3
5799f56e101a3ec4.jpg сохранено в класс 3
57aa85070c8e6a81.jpg сохранено в класс 3
57dca93ee0898e5a.jpg сохранено в класс 2
58b2228cb7e66614.jpg сохранено в класс 3
58e5380fe3e930f1.jpg сохранено в класс 2
59bc707325fd5f4a.jpg сохранено в класс 3
59c6a0c1c1cc4e3d.jpg сохранено в класс 3
59f0f47ae3dab3d3.jpg сохранено в класс 3
5a243080d185fe8e.jpg сохранено в класс 3
5a350babe99d3b88

# Код разметки символов по положению в трафарете

## 3 класса: 
0 - трафарет нужно сдвинуть влево
1 - символ в центре трафарета, ничего не меняем
2 - трафарет нужно сдвинуть вправо

In [ ]:
import os
import cv2

# Папка с изображениями для классификации
source_folder = r"F:\grozny\beeline\train_borders\fragments"

# Папка для сохранения классифицированных изображений
destination_folder = r"F:\grozny\beeline\train_borders\recog"
class_0_folder = os.path.join(destination_folder, '0')
class_1_folder = os.path.join(destination_folder, '1')
class_2_folder = os.path.join(destination_folder, '2')

# Создание папок для каждого класса, если они не существуют
os.makedirs(class_0_folder, exist_ok=True)
os.makedirs(class_1_folder, exist_ok=True)
os.makedirs(class_2_folder, exist_ok=True)

# Размер для масштабирования изображений
resize_dim = (64, 64)

# Получение списка всех файлов в исходной папке
files = [f for f in os.listdir(source_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

for filename in files:
    img_path = os.path.join(source_folder, filename)
    img = cv2.imread(img_path)
    
    if img is None:
        print(f"Не удалось открыть изображение {img_path}")
        continue

    # Масштабирование изображения
    resized_img = cv2.resize(img, resize_dim)

    # Показ изображения
    cv2.imshow('Image', resized_img)

    key = cv2.waitKey(0)  # Ожидание нажатия клавиши

    # Копирование изображения в соответствующую папку в зависимости от нажатой клавиши
    if key == ord('q'):
        destination_path = os.path.join(class_0_folder, filename)
        cv2.imwrite(destination_path, resized_img)
        print(f"{filename} сохранено в класс 0")
    elif key == ord('w'):
        destination_path = os.path.join(class_1_folder, filename)
        cv2.imwrite(destination_path, resized_img)
        print(f"{filename} сохранено в класс 1")
    elif key == ord('e'):
        destination_path = os.path.join(class_2_folder, filename)
        cv2.imwrite(destination_path, resized_img)
        print(f"{filename} сохранено в класс 2")
    else:
        print(f"{filename} не классифицировано, пропускается")

    cv2.destroyAllWindows()  # Закрыть окно изображения

print("Классификация завершена!")


002d28e4d47606a2_pos1.png сохранено в класс 1
002d28e4d47606a2_pos2.png сохранено в класс 1
002d28e4d47606a2_pos3.png сохранено в класс 1
002d28e4d47606a2_pos4.png сохранено в класс 2
002d28e4d47606a2_pos5.png сохранено в класс 2
002d28e4d47606a2_pos6.png сохранено в класс 2
002d28e4d47606a2_pos7.png не классифицировано, пропускается
002d28e4d47606a2_pos8.png сохранено в класс 1
002d28e4d47606a2_pos9.png сохранено в класс 1
0096f97c017f7545_pos1.png сохранено в класс 1
0096f97c017f7545_pos2.png сохранено в класс 0
0096f97c017f7545_pos3.png сохранено в класс 0
0096f97c017f7545_pos4.png сохранено в класс 0
0096f97c017f7545_pos5.png сохранено в класс 0
0096f97c017f7545_pos6.png сохранено в класс 0
0096f97c017f7545_pos7.png сохранено в класс 1
0096f97c017f7545_pos8.png сохранено в класс 1
00c980fd3135c4af_pos1.png сохранено в класс 1
00c980fd3135c4af_pos2.png сохранено в класс 1
00c980fd3135c4af_pos3.png сохранено в класс 2
00c980fd3135c4af_pos4.png сохранено в класс 1
00c980fd3135c4af_pos

0564781d52c0ed11_pos8.png сохранено в класс 0
0564781d52c0ed11_pos9.png сохранено в класс 1
05afe0f65aff91ee_pos1.png сохранено в класс 1
05afe0f65aff91ee_pos2.png сохранено в класс 1
05afe0f65aff91ee_pos3.png сохранено в класс 1
05afe0f65aff91ee_pos4.png сохранено в класс 1
05afe0f65aff91ee_pos5.png сохранено в класс 1
05afe0f65aff91ee_pos6.png сохранено в класс 1
05afe0f65aff91ee_pos7.png сохранено в класс 1
05afe0f65aff91ee_pos8.png сохранено в класс 1
05afe0f65aff91ee_pos9.png сохранено в класс 2
05be91f5032c0f79_pos1.png сохранено в класс 1
05be91f5032c0f79_pos2.png сохранено в класс 1
05be91f5032c0f79_pos3.png сохранено в класс 2
05be91f5032c0f79_pos4.png сохранено в класс 2
05be91f5032c0f79_pos5.png сохранено в класс 2
05be91f5032c0f79_pos6.png сохранено в класс 2
05be91f5032c0f79_pos7.png не классифицировано, пропускается
05be91f5032c0f79_pos8.png сохранено в класс 1
05be91f5032c0f79_pos9.png сохранено в класс 1
060907044a5815a7_pos1.png сохранено в класс 1
060907044a5815a7_pos

0ac22544b27be4ef_pos5.png сохранено в класс 1
0ac22544b27be4ef_pos6.png сохранено в класс 1
0ac22544b27be4ef_pos7.png сохранено в класс 1
0ac22544b27be4ef_pos8.png сохранено в класс 1
0ac22544b27be4ef_pos9.png сохранено в класс 1
0b288ece14de5bb5_pos1.png сохранено в класс 1
0b288ece14de5bb5_pos2.png сохранено в класс 0
0b288ece14de5bb5_pos3.png сохранено в класс 0
0b288ece14de5bb5_pos4.png сохранено в класс 0
0b288ece14de5bb5_pos5.png сохранено в класс 0
0b288ece14de5bb5_pos6.png сохранено в класс 0
0b288ece14de5bb5_pos7.png сохранено в класс 0
0b288ece14de5bb5_pos8.png сохранено в класс 1
0b59a6532708ac55_pos1.png сохранено в класс 0
0b59a6532708ac55_pos2.png сохранено в класс 1
0b59a6532708ac55_pos3.png сохранено в класс 1
0b59a6532708ac55_pos4.png сохранено в класс 1
0b59a6532708ac55_pos5.png сохранено в класс 1
0b59a6532708ac55_pos6.png сохранено в класс 1
0b59a6532708ac55_pos7.png сохранено в класс 1
0b59a6532708ac55_pos8.png сохранено в класс 1
0b5e5490607902fe_pos1.png сохранен

10065108a94db3f1_pos1.png сохранено в класс 1
10065108a94db3f1_pos2.png сохранено в класс 1
10065108a94db3f1_pos3.png сохранено в класс 1
10065108a94db3f1_pos4.png сохранено в класс 1
10065108a94db3f1_pos5.png сохранено в класс 2
10065108a94db3f1_pos6.png сохранено в класс 2
10065108a94db3f1_pos7.png сохранено в класс 2
10065108a94db3f1_pos8.png сохранено в класс 2
100a09c3bd5ff698_pos1.png сохранено в класс 1
100a09c3bd5ff698_pos2.png сохранено в класс 1
100a09c3bd5ff698_pos3.png сохранено в класс 1
100a09c3bd5ff698_pos4.png сохранено в класс 1
100a09c3bd5ff698_pos5.png сохранено в класс 2
100a09c3bd5ff698_pos6.png сохранено в класс 2
100a09c3bd5ff698_pos7.png не классифицировано, пропускается
100a09c3bd5ff698_pos8.png сохранено в класс 1
100a09c3bd5ff698_pos9.png сохранено в класс 2
10942eb0c5a1bed6_pos1.png сохранено в класс 2
10942eb0c5a1bed6_pos2.png сохранено в класс 2
10942eb0c5a1bed6_pos3.png сохранено в класс 2
10942eb0c5a1bed6_pos4.png сохранено в класс 2
10942eb0c5a1bed6_pos

140a4d756cd98ae2_pos7.png сохранено в класс 1
140a4d756cd98ae2_pos8.png сохранено в класс 1
1434c026ca53115c_pos1.png сохранено в класс 1
1434c026ca53115c_pos2.png сохранено в класс 1
1434c026ca53115c_pos3.png сохранено в класс 1
1434c026ca53115c_pos4.png сохранено в класс 1
1434c026ca53115c_pos5.png сохранено в класс 1
1434c026ca53115c_pos6.png сохранено в класс 1
1434c026ca53115c_pos7.png сохранено в класс 1
1434c026ca53115c_pos8.png сохранено в класс 1
1434c026ca53115c_pos9.png сохранено в класс 2
148dd899509acc4b_pos1.png сохранено в класс 1
148dd899509acc4b_pos2.png сохранено в класс 1
148dd899509acc4b_pos3.png сохранено в класс 1
148dd899509acc4b_pos4.png сохранено в класс 1
148dd899509acc4b_pos5.png сохранено в класс 1
148dd899509acc4b_pos6.png сохранено в класс 1
148dd899509acc4b_pos7.png не классифицировано, пропускается
148dd899509acc4b_pos8.png сохранено в класс 1
14d08b85ce37009c_pos1.png сохранено в класс 0
14d08b85ce37009c_pos2.png сохранено в класс 1
14d08b85ce37009c_pos

196394e57850da4b_pos2.png сохранено в класс 2
196394e57850da4b_pos3.png сохранено в класс 2
196394e57850da4b_pos4.png сохранено в класс 2
196394e57850da4b_pos5.png сохранено в класс 2
196394e57850da4b_pos6.png сохранено в класс 0
196394e57850da4b_pos7.png не классифицировано, пропускается
196394e57850da4b_pos8.png сохранено в класс 1
196394e57850da4b_pos9.png сохранено в класс 1
1966f5a348f63b0e_pos1.png сохранено в класс 1
1966f5a348f63b0e_pos2.png не классифицировано, пропускается
1966f5a348f63b0e_pos3.png сохранено в класс 1
1966f5a348f63b0e_pos4.png сохранено в класс 1
1966f5a348f63b0e_pos5.png сохранено в класс 1
1966f5a348f63b0e_pos6.png сохранено в класс 1
1966f5a348f63b0e_pos7.png сохранено в класс 2
1966f5a348f63b0e_pos8.png сохранено в класс 2
1a19cd008b0bde6c_pos1.png сохранено в класс 1
1a19cd008b0bde6c_pos2.png сохранено в класс 1
1a19cd008b0bde6c_pos3.png сохранено в класс 1
1a19cd008b0bde6c_pos4.png сохранено в класс 1
1a19cd008b0bde6c_pos5.png сохранено в класс 1
1a19cd

1d16849753b20612_pos4.png сохранено в класс 1
1d16849753b20612_pos5.png сохранено в класс 1
1d16849753b20612_pos6.png сохранено в класс 2
1d16849753b20612_pos7.png сохранено в класс 2
1d16849753b20612_pos8.png сохранено в класс 2
1d16849753b20612_pos9.png сохранено в класс 2
1d2f67c87c14be71_pos1.png сохранено в класс 1
1d2f67c87c14be71_pos2.png сохранено в класс 1
1d2f67c87c14be71_pos3.png сохранено в класс 1
1d2f67c87c14be71_pos4.png сохранено в класс 1
1d2f67c87c14be71_pos5.png сохранено в класс 1
1d2f67c87c14be71_pos6.png сохранено в класс 1
1d2f67c87c14be71_pos7.png сохранено в класс 1
1d2f67c87c14be71_pos8.png сохранено в класс 1
1d2f67c87c14be71_pos9.png сохранено в класс 1
1d3b00cfd6c20193_pos1.png сохранено в класс 0
1d3b00cfd6c20193_pos2.png сохранено в класс 0
1d3b00cfd6c20193_pos3.png сохранено в класс 0
1d3b00cfd6c20193_pos4.png сохранено в класс 1
1d3b00cfd6c20193_pos5.png сохранено в класс 1
1d3b00cfd6c20193_pos6.png сохранено в класс 1
1d3b00cfd6c20193_pos7.png сохранен

21f6d8e94e103cc0_pos6.png сохранено в класс 0
21f6d8e94e103cc0_pos7.png не классифицировано, пропускается
21f6d8e94e103cc0_pos8.png сохранено в класс 0
21f6d8e94e103cc0_pos9.png сохранено в класс 1
22106630cefbab0c_pos1.png сохранено в класс 1
22106630cefbab0c_pos2.png сохранено в класс 1
22106630cefbab0c_pos3.png сохранено в класс 1
22106630cefbab0c_pos4.png сохранено в класс 1
22106630cefbab0c_pos5.png сохранено в класс 1
22106630cefbab0c_pos6.png сохранено в класс 1
22106630cefbab0c_pos7.png сохранено в класс 1
22106630cefbab0c_pos8.png сохранено в класс 1
22106630cefbab0c_pos9.png сохранено в класс 1
22b575e9b2dc748e_pos1.png сохранено в класс 0
22b575e9b2dc748e_pos2.png сохранено в класс 0
22b575e9b2dc748e_pos3.png сохранено в класс 1
22b575e9b2dc748e_pos4.png сохранено в класс 1
22b575e9b2dc748e_pos5.png сохранено в класс 0
22b575e9b2dc748e_pos6.png сохранено в класс 1
22b575e9b2dc748e_pos7.png сохранено в класс 1
22b575e9b2dc748e_pos8.png сохранено в класс 1
22b575e9b2dc748e_pos

# Код для сохранения символов из трафарета

## Использовался для подготовки изображений к обучению сети, предсказывающей положение символа в трафарете

In [21]:
import cv2
from PIL import Image
import os

def extract_fragments(image_path, template, output_dir):
    # Загрузить изображение
    image = cv2.imread(image_path)
    height, width, _ = image.shape
    print(height, width)
    if height < 100:
        width = int(width * (100 / height))
        height = 100
        
    print(height, width)
    image = cv2.resize(image, (width, height))
    # Убедиться, что директория для сохранения фрагментов существует
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Извлечь фрагменты
    for region in template:
        pos = region['pos']
        p1 = (int(region['p1'][0] * width), int(region['p1'][1] * height))
        p2 = (int(region['p2'][0] * width), int(region['p2'][1] * height))
        
        fragment = image[p1[1]:p2[1], p1[0]:p2[0]]
        
        # Сохранить фрагмент как изображение
        fragment_path = os.path.join(output_dir, f'fragment_{pos}.png')
        cv2.imwrite(fragment_path, fragment)
        print(f"Fragment {pos} saved at {fragment_path}")

# Пример использования
names = os.listdir(r'F:\grozny\beeline\train_borders\align')
image_path = r'F:\grozny\beeline\train_borders\align\\' + names[7]
output_dir = r'F:\grozny\beeline\train_borders\fragments'  # Директория для сохранения фрагментов

two_digit_region_template = [
    {'pos': 1, 'p1': (0.05, 0.250), 'p2': (0.16, 0.92)},
    {'pos': 2, 'p1': (0.16, 0.100), 'p2': (0.27, 0.92)},
    {'pos': 3, 'p1': (0.265, 0.100), 'p2': (0.375, 0.92)},
    {'pos': 4, 'p1': (0.37, 0.100), 'p2': (0.48, 0.92)},
    {'pos': 5, 'p1': (0.475, 0.250), 'p2': (0.585, 0.92)},
    {'pos': 6, 'p1': (0.58, 0.250), 'p2': (0.69, 0.92)},
    {'pos': 7, 'p1': (0.71, 0.05), 'p2': (0.795, 0.7)},
    {'pos': 8, 'p1': (0.79, 0.05), 'p2': (0.875, 0.7)},
    {'pos': 9, 'p1': (0.87, 0.05), 'p2': (0.96, 0.7)}
]

extract_fragments(image_path, two_digit_region_template, output_dir)


13 65
100 500
Fragment 1 saved at F:\grozny\beeline\train_borders\fragments\fragment_1.png
Fragment 2 saved at F:\grozny\beeline\train_borders\fragments\fragment_2.png
Fragment 3 saved at F:\grozny\beeline\train_borders\fragments\fragment_3.png
Fragment 4 saved at F:\grozny\beeline\train_borders\fragments\fragment_4.png
Fragment 5 saved at F:\grozny\beeline\train_borders\fragments\fragment_5.png
Fragment 6 saved at F:\grozny\beeline\train_borders\fragments\fragment_6.png
Fragment 7 saved at F:\grozny\beeline\train_borders\fragments\fragment_7.png
Fragment 8 saved at F:\grozny\beeline\train_borders\fragments\fragment_8.png
Fragment 9 saved at F:\grozny\beeline\train_borders\fragments\fragment_9.png
